In [25]:
import torch
import torchvision
import json
import pandas as pd
from healthpricefinder.utils import iterate_for_df
import ijson
import os


In [2]:
# !pip install -e ../../.

In [3]:
df_row_base = {"negotiation_arrangement": [None], "name": [None], "billing_code_type": [None], "billing_code_type_version": [None], "billing_code": [None],
              "description": [None], "negotiated_rate": [None], "expiration": [None], "provider_references": [None], "negotiated_type": [None], "billing_class": [None]}


## Extract provider reference id to NPI relationship

In [32]:
provider_reference = next(f)

In [44]:
prov_dict = provider_reference[:3090816]
prov_dict += "}"

In [64]:
providers = json.loads(prov_dict)["provider_references"]
provider_df = pd.DataFrame()
for p in providers:
    provider_dict = {"provider_id": [], "npi_list": [], "tin": [], "tin_type": []}
    provider_dict["provider_id"].append(p["provider_group_id"])
    provider_dict["npi_list"].append(str(p["provider_groups"][0]["npi"]))
    provider_dict["tin"].append(p["provider_groups"][0]["tin"]["value"])
    provider_dict["tin_type"].append(p["provider_groups"][0]["tin"]["type"])
    provider_row = pd.DataFrame.from_dict(provider_dict)
    provider_df = pd.concat([provider_df, provider_row])
provider_df = provider_df.reset_index(drop=True)

In [65]:
provider_df.to_csv("../../data/aetna_piecewise/provider_references.csv")

## Extract price per procedure csv

In [4]:
def extract_dictlike(s):
    start_idx = s.find("negotiation_arrangement")
    end_idx = s.find("negotiated_rates")
    first_dictlike = s[start_idx-1:end_idx-2]
    first_dict = "{" + first_dictlike + "}"
    first_dict = json.loads(first_dict)
    return first_dict

In [23]:
def update_df(fileobj, df_row_base, prices_df, num_iters=1000):
    for i in range(num_iters):
        line = next(f)
        if line.find("negotiation_arrangement") != -1:
            procedure_row = extract_dictlike(line)
            df_row_base.update(procedure_row)
        else:
            start_idx = line.find("provider_references")
            line = line[start_idx:]
            dictlike = '{"'+ line
            prices = json.loads(dictlike)
            df_row_base["provider_references"] = str(prices["provider_references"])
            df_row_base["negotiated_rate"] = prices["negotiated_prices"][0]["negotiated_rate"]
            df_row_base["expiration_date"] = prices["negotiated_prices"][0]["expiration_date"]
            df_row_base["billing_class"] = prices["negotiated_prices"][0]["billing_class"]
            df = pd.DataFrame(df_row_base)
            prices_df = pd.concat([prices_df, df])
    return prices_df

In [21]:
len(f.readlines())

9523394

In [30]:
df_row_base = {"negotiation_arrangement": [None], "name": [None], "billing_code_type": [None], "billing_code_type_version": [None], "billing_code": [None],
              "description": [None], "negotiated_rate": [None], "expiration_date": [None], "provider_references": [None], "negotiated_type": [None], "billing_class": [None]}

df_row = dict(df_row_base)
prices_df = pd.DataFrame()
silver_path = "../../data/2022-09-05_bf1ee0a7-5440-47e6-a69f-c376fb6b01a7_Aetna-Life-insurance-Company.json"
f = open(silver_path, 'r+')
num_pieces = 100
num_iters = 10000
for k in range(num_pieces):
    prices_df_intermediate = update_df(f, df_row_base, prices_df, num_iters)
    prices_df = pd.concat([prices_df, prices_df_intermediate])
    prices_df.reset_index(drop=True).to_csv(f"../../data/aetna_piecewise/procedure_costs_{(k+1)*num_iters}.csv")
    prev_file = f"../../data/aetna_piecewise/procedure_costs_{(k)*num_iters}.csv"
    if os.path.exists(prev_file):
        os.remove(prev_file)
    

KeyboardInterrupt: 

In [32]:
num_iters

10000

{'provider_references': [372414],
 'negotiated_prices': [{'negotiated_type': 'per diem',
   'negotiated_rate': 1161.0,
   'expiration_date': '9999-12-31',
   'service_code': ['21',
    '31',
    '32',
    '33',
    '34',
    '51',
    '54',
    '55',
    '56',
    '61'],
   'billing_class': 'institutional'}]}

In [132]:
print(line)

"provider_references":[372414],"negotiated_prices":[{"negotiated_type": "per diem","negotiated_rate": 1161.00,"expiration_date": "9999-12-31","service_code": ["21", "31", "32", "33", "34", "51", "54", "55", "56", "61"],"billing_class": "institutional"}]}



In [21]:
df_row_base = {"negotiation_arrangement": [None], "name": [None], "billing_code_type": [None], "billing_code_type_version": [None], "billing_code": [None],
              "description": [None], "negotiated_rate": [None], "expiration_date": [None], "provider_references": [None], "negotiated_type": [None], "billing_class": [None]}

df_row = dict(df_row_base)

In [22]:
df_row.keys()

dict_keys(['negotiation_arrangement', 'name', 'billing_code_type', 'billing_code_type_version', 'billing_code', 'description', 'negotiated_rate', 'expiration_date', 'provider_references', 'negotiated_type', 'billing_class'])

In [23]:
df_row = df_row_base.copy()
procedure_dict = dict(df_row)
iter_step_size = 1000000

## Extract the disease and pricing information for 10 iter cycles

In [63]:
iter_step_size = 1000000
for i in range(10):
    main = iterate_for_df(parser, df_row, iter_step_size)
    main.to_csv(f"../../data/aetna_piecewise/california_sept_{i}.csv")

UnboundLocalError: local variable 'provider_references_list' referenced before assignment